In [1]:
import sys
import os
import importlib
import time
import numpy as np
import random
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer #计算tfidf
from sklearn.feature_extraction.text import CountVectorizer  #计算df
from sklearn.feature_extraction.text import TfidfVectorizer  #“一步到位”
from gensim.models import word2vec, Word2Vec
import gensim.models
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.externals import joblib
from sklearn import metrics

id_num = []
word_article = []
words_article = []
label = []
X_train = []
y_train = []
X_test = []
y_test = []

load_start = time.time()
importlib.reload(sys)
with open('train_set.csv','r',encoding = 'utf-8') as f:
    lines = f.readlines()[1:]
    random.shuffle(lines)
    for line in lines[1:20000]:
        line = line.split(',')
        X_train.append(line[2])
        y_train.append(line[3])
    for line in lines[950000:]:
        line = line.split(',')
        X_test.append(line[2])
        y_test.append(line[3])
print("Time consumption on loading:",time.time() - load_start)

train_start = time.time()
w2v_size = 256
w2v_model = Word2Vec.load('./word2vec/w2v_by_fullset.model')
def tran_X(X_list):
    X = [[] for i in range(len(X_list))]
    for i in range(0,len(X_list)):
        j = X_list[i].split(' ') #把str分成词list
        X[i] += j 
#     print(X)
    X_vectors = np.zeros((len(X),w2v_size))
    i = 0
    for words in X:
        j = 0
        for word in words:
            if word in w2v_model.wv.vocab:
                X_vectors[i] += w2v_model[word] 
                j += 1
        X_vectors[i] /= j
        i += 1
    print("shape:",X_vectors.shape)
    
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    X_vectors = imp.fit_transform(X_vectors)
#     for i in range(len(X_vectors)): 
#         for j in range(len(X_vectors[i])):
#             if np.isnan(X_vectors[i][j]): #去除NaN
#                 X_vectors[i][j] = 0
    
    X_vectors = preprocessing.normalize(X_vectors, norm='l2', axis=1, copy=True) #正则化 74 68/73 70
#     ss=StandardScaler() 
#     X_vectors=ss.fit_transform(X_vectors) #70 67/70 68
#     X_vectors = preprocessing.scale(X_vectors) #标准化 69 67/70 66
#     X_vectors = preprocessing.MinMaxScaler().fit_transform(X_vectors) #归一化 71 65/72 65
#     model_vectors = np.transpose(model_vectors) #转置
    return X_vectors

X_train = tran_X(X_train)
print("Time consumption on feature_training:",time.time() - train_start)
train_start = time.time()
# clf = svm.LinearSVC(C=10) #C=1/10：72 73已经是最好水平
clf = svm.SVC(kernel='rbf', C=4, gamma=2) #C=4 gamma=2：77 76已经是最好水平
clf = svm.SVC(kernel='rbf', C=4, gamma=2)
# clf = SGDClassifier(loss="modified_huber", penalty="elasticnet", alpha=0.0001, n_jobs=-1)# modified_huber;  penalty="elasticnet"不然l1 l2recall上不去？
# clf = SGDClassifier(loss="log", penalty="elasticnet", alpha=0.000005, n_jobs=-1)
# parameters = {#'loss':('log', 'hinge','modified_huber'),
#               #'alpha':[0.000002, 0.00001, 0.0001],
#               #'penalty':['elasticnet','l2']
#                 'C':[1,2,4,8],
#                 'gamma':[1,2,4,8]
#             }
# grid_search = GridSearchCV(clf, parameters, n_jobs=-1, cv=3)
# grid_search.fit(X_train, y_train)
# best_parameters = dict(grid_search.best_estimator_.get_params())#get实例中的最优参数
# for param_name in sorted(parameters.keys()):
#     print("\t%s: %r" % (param_name, best_parameters[param_name])) #输出参数结果
clf.fit(X_train, y_train)
# joblib.dump(clf, 'lineSVM_20000__.m')
print("Time consumption on clf_training:",time.time() - train_start)

test_start = time.time()
# clf = joblib.load('lineSVM_20000__.m')
prediction = clf.predict(X_train) #训练集上准确率
print(metrics.classification_report(y_train,prediction))
X_test = tran_X(X_test)
prediction = clf.predict(X_test) #测试集上准确率
print(metrics.classification_report(y_test,prediction))
# print(metrics.confusion_matrix(y_test,prediction))
print("Time consumption on testing:",time.time() - test_start)

Time consumption on loading: 2.778332233428955


/home/amax/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:67: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


shape: (79999, 256)
Time consumption on feature_training: 594.886815071106
Time consumption on clf_training: 758.9142446517944
             precision    recall  f1-score   support

         1
       0.78      0.70      0.74      4232
        10
       0.82      0.81      0.81      3829
        11
       0.76      0.80      0.78      2748
        12
       0.81      0.80      0.81      4157
        13
       0.82      0.87      0.85      6159
        14
       0.81      0.88      0.85      5257
        15
       0.95      0.93      0.94      5862
        16
       0.86      0.72      0.79      2539
        17
       0.83      0.83      0.83      2447
        18
       0.92      0.92      0.92      5554
        19
       0.70      0.75      0.73      4363
         2
       0.88      0.88      0.88      2266
         3
       0.92      0.91      0.91      6502
         4
       0.91      0.91      0.91      2964
         5
       0.94      0.87      0.90      1857
         6
       0.97  